In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

In [5]:
gains = [2,5,4]
buffer_error = np.zeros((len(gains),10))

class Controller():

    def sum_err(buffer_error):
        sum_err = []
        for i in range(len(buffer_error)):
            sum_err.append(np.sum(buffer_error))
        sum_err = np.array(sum_err)
        return sum_err

    def tau(error):
        
        buffer_error = np.hstack((buffer_error[:,1:],np.transpose([error])))
        tau = gains[:,0]*error + gains[:,1]*calculate_sum_error()
        return tau

In [7]:
# SCARA 

# Initialization
g = 9.8 
l_1 = 1
l_2 = 1
l_3 = 1
d = 1
gains = [[10,0,0],[20,0,0],[10,0,0]]
m_1 = 1
m_2 = 1
m_3 = 1
x = np.zeros(3)
x_dot = np.zeros(3)
x_ddot = np.zeros(3)
tau_1 = 0 
tau_2 = 0
F = 0 # Force for prismatic joint
J_1 = (1/3)*m_1*l_1**2
J_2 = (1/3)*m_2*l_2**2
J_3 = (1/3)*m_3*l_3**2

class SCARA():

    def inv_kin(self, end_eff):  # Inverse Kinematics
        [x_e,y_e,z_e] = end_eff
        if np.sqrt(x_e**2 + y_e**2)> l_2 + l_1:
            print("Robot out of Workspace!")
        else: 
            def inv(x):
                return [l_1*np.cos(x[0]) + l_2*np.cos(x[0]+x[1]) - x_e,l_1*np.sin(x[0]) + l_2*np.sin(x[0]+x[1]) - y_e,d - x[2] - z_e]

            joint_var = fsolve(self.inv_kin,[1,1,1]) 
            q1,q2,d = joint_var # Joint Variables
    
    def for_kin(l_1,l_2,x,d): # Forward Kinematics - Check!

        x_e = l_1*np.cos(x[0]) + l_2*np.cos(x[0] + x[1])
        y_e = l_1*np.sin(x[0]) + l_2*np.sin(x[0] + x[1])
        z_e = d - x[2]

        [x_end,y_end,z_end] = [x_e,y_e,z_e]

        return x_e, y_e, z_e
    
    def dynamics(self,dt,x_des): # Dynamics reffered from a research paper

        err = x_des - x

        # Note that theta, phi and psi are just constants and does not represent angles
        
        theta = J1 + l_1**2*(m_1/4 + m_2 + m_3)
        phi = J2 + J3 + l_2**2*(m_2/4 + m_3) 
        psi = l_1*l_2*m_3 + l_1 * l_2/2 * m_2

        tau_1,tau_2,F = controller.tau(error)

        u_1 = -psi*np.sin(x[1])*x_dot[1]*x_dot[0] - psi*np.sin(x[1])*(x_dot[1] + x_dot[0])*x_dot[1] + tau_1
        u_2 = psi*np.sin(x[1])*x_dot[0]**2 + tau_2
        u_3 = m_3*g + F

        M = np.array([[theta + phi + 2*psi*np.cos(x[1]), phi + 2*psi*np.cos(x[1]), 0],[phi + 2*psi*np.cos(x[1]), phi, 0],[0, 0, m_3]])
        
        C = np.array([[-psi*np.sin(x[1])*x_dot[1], -psi*np.sin(x[1])*(x_dot[1] + x_dot[0]), 0],[psi*np.sin(x[1])*x_dot[0], 0, 0],[0, 0, 0]])
        
        G = np.transpose(np.array([0, 0, m_3*g]))

        U = np.transpose(np.array([u_1, u_2,u_3]))

        K = U - np.matmul(C, np.transpose(x_dot)) - G

        d2y_dt2 = np.matmul(np.linalg.inv(M), K)


        x_dot = x_dot + d2y_dt2*dt 
        x = x_dot*dt + 1/2*d2y_dt2*dt**2
    
    def tracking(self, track_pts):

        x_des,y_des,z_des = track_pts

        angles = self.inv_kin([x_des,y_des,z_des])
  
        max_time = 20 
        time_steps = 1000
        t = np.linspace(0,max_time,num=num_steps )
        y = []
        y_inv = []
        y.append(x)
        y_inv.append([x_end,y_end,z_end])

        for i in range(len(t)):
            dynamics(max_time/num_steps,angles)
            y.append(x)
            y_inv.append([x_end,y_end,z_end])
            
        return np.array(y),np.array(y_inv)

s = SCARA()
x_des = -1.3
y_des = -0.6
z_des = 1.6

des_pt = [x_des,y_des,z_des]
y,y_inv = s.tracking(des_pt)
plt.figure(1)
plt.plot(y_inv[:,2])
plt.plot(y_inv[:,0])
plt.plot(y_inv[:,1])

plt.show()

RecursionError: maximum recursion depth exceeded